In [1]:
import neattext.functions as nfx
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Dense
import nltk
from sklearn.model_selection import StratifiedKFold
from googletrans import Translator
import random

translator = Translator()
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gabi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
data = pd.read_csv("C:/Users/Gabi/PycharmProjects/multimodal-emotion-recognition/text_data.csv")

In [8]:
target_samples = 1000

# Function to perform back-translation
def back_translate(text):
    try:
        translation = translator.translate(text, src="en", dest="fr")
        back_translation = translator.translate(translation.text, src="fr", dest="en")
        return back_translation.text
    except:
        return text


hap_data = data[data['emotion'] == 'hap']
# Apply back-translation to "hap" emotion sentences
augmented_sentences = []
# Iterate through each sentence in the dataset
# Randomly select sentences and apply back-translation
for _ in range(target_samples):
    random_index = random.choice(hap_data.index)
    original_sentence = hap_data.loc[random_index, 'sentence']
    back_translated_sentence = back_translate(original_sentence)
    augmented_sentences.append(back_translated_sentence)

# Create a new DataFrame with the augmented sentences
augmented_data = pd.DataFrame({'emotion': 'hap', 'sentence': augmented_sentences})

# Combine the augmented data with the original data
combined_data = pd.concat([data, augmented_data], ignore_index=True)

In [9]:
target_samples = 500

ang_data = data[data['emotion'] == 'ang']

augmented_sentences = []


for _ in range(target_samples):
    random_index = random.choice(ang_data.index)
    original_sentence = ang_data.loc[random_index, 'sentence']
    back_translated_sentence = back_translate(original_sentence)
    augmented_sentences.append(back_translated_sentence)

augmented_data = pd.DataFrame({'emotion': 'ang', 'sentence': augmented_sentences})

combined_data = pd.concat([combined_data, augmented_data], ignore_index=True)

In [10]:
target_samples = 500

sad_data = data[data['emotion'] == 'sad']

augmented_sentences = []


for _ in range(target_samples):
    random_index = random.choice(sad_data.index)
    original_sentence = sad_data.loc[random_index, 'sentence']
    back_translated_sentence = back_translate(original_sentence)
    augmented_sentences.append(back_translated_sentence)

augmented_data = pd.DataFrame({'emotion': 'sad', 'sentence': augmented_sentences})

combined_data = pd.concat([combined_data, augmented_data], ignore_index=True)

In [11]:
target_samples = 1000

exc_data = data[data['emotion'] == 'exc']

augmented_sentences = []


for _ in range(target_samples):
    random_index = random.choice(exc_data.index)
    original_sentence = exc_data.loc[random_index, 'sentence']
    back_translated_sentence = back_translate(original_sentence)
    augmented_sentences.append(back_translated_sentence)

augmented_data = pd.DataFrame({'emotion': 'exc', 'sentence': augmented_sentences})

combined_data = pd.concat([combined_data, augmented_data], ignore_index=True)

In [12]:
combined_data['Clean_Text'] = combined_data['sentence'].apply(nfx.remove_userhandles)
combined_data['Clean_Text'] = combined_data['Clean_Text'].apply(nfx.remove_stopwords)

combined_data['emotion'].value_counts()

emotion
exc    2041
fru    1849
neu    1708
ang    1603
hap    1595
sad    1584
Name: count, dtype: int64

In [13]:
cv = CountVectorizer(max_features=5000, ngram_range=(1, 3))
data_cv = cv.fit_transform(combined_data['Clean_Text']).toarray()

In [14]:
label_encoder = preprocessing.LabelEncoder()
encoded_labels = label_encoder.fit_transform(combined_data['emotion'])

In [15]:
def create_model():
    model = Sequential()
    model.add(Dense(12, input_shape=(data_cv.shape[1],), activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# accuracy_scores = []
#
# for train_idx, test_idx in cv.split(data_cv, encoded_labels):
#     X_train, X_test = data_cv[train_idx], data_cv[test_idx]
#     y_train, y_test = encoded_labels[train_idx], encoded_labels[test_idx]
#
#     model = create_model()
#     model.fit(X_train, y_train, epochs=200, batch_size=10, verbose=1)
#
#     y_pred_probs = model.predict(X_test)
#     y_test_pred = np.argmax(y_pred_probs, axis=1)
#
#     test_accuracy = accuracy_score(y_test, y_test_pred)
#     print("Test Accuracy:", test_accuracy)
#
#     test_classification_rep = classification_report(y_test, y_test_pred, target_names=label_encoder.classes_)
#     print("Test Classification Report:\n", test_classification_rep)
#
#     y_pred = np.argmax(y_pred_probs, axis=1)
#     accuracy = accuracy_score(y_test, y_pred)
#     accuracy_scores.append(accuracy)

In [ ]:
# print("Cross-Validation Scores:", accuracy_scores)
# print("Mean CV Score:", np.mean(accuracy_scores))
# print("Standard Deviation:", np.std(accuracy_scores))

In [16]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_accuracies = []
test_accuracies = []
test_classification_reports = []

for train_idx, test_idx in cv.split(data_cv, encoded_labels):
    X_train, X_test = data_cv[train_idx], data_cv[test_idx]
    y_train, y_test = encoded_labels[train_idx], encoded_labels[test_idx]

    model = create_model()
    model.fit(X_train, y_train, epochs=200, batch_size=10, verbose=1)

    # Calculate train accuracy
    y_train_pred_probs = model.predict(X_train)
    y_train_pred = np.argmax(y_train_pred_probs, axis=1)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracies.append(train_accuracy)

    # Calculate test accuracy
    y_test_pred_probs = model.predict(X_test)
    y_test_pred = np.argmax(y_test_pred_probs, axis=1)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_accuracies.append(test_accuracy)

    # Generate classification report for test data
    test_classification_rep = classification_report(y_test, y_test_pred, target_names=label_encoder.classes_)
    test_classification_reports.append(test_classification_rep)

print("Train Accuracies:", train_accuracies)
print("Mean Train Accuracy:", np.mean(train_accuracies))
print("Train Accuracy Standard Deviation:", np.std(train_accuracies))

# Print or analyze test accuracies
print("Test Accuracies:", test_accuracies)
print("Mean Test Accuracy:", np.mean(test_accuracies))
print("Test Accuracy Standard Deviation:", np.std(test_accuracies))

# Print or analyze test classification reports
for fold_idx, report in enumerate(test_classification_reports):
    print(f"Classification Report for Fold {fold_idx + 1}:\n{report}")

Epoch 1/200
831/831 [==============================] - 1s 812us/step - loss: 1.5500 - accuracy: 0.3560
Epoch 2/200
831/831 [==============================] - 1s 781us/step - loss: 1.0937 - accuracy: 0.5845
Epoch 3/200
831/831 [==============================] - 1s 784us/step - loss: 0.8960 - accuracy: 0.6625
Epoch 4/200
831/831 [==============================] - 1s 773us/step - loss: 0.7776 - accuracy: 0.7060
Epoch 5/200
831/831 [==============================] - 1s 770us/step - loss: 0.7005 - accuracy: 0.7335
Epoch 6/200
831/831 [==============================] - 1s 780us/step - loss: 0.6480 - accuracy: 0.7512
Epoch 7/200
831/831 [==============================] - 1s 794us/step - loss: 0.6041 - accuracy: 0.7653
Epoch 8/200
831/831 [==============================] - 1s 790us/step - loss: 0.5709 - accuracy: 0.7809
Epoch 9/200
831/831 [==============================] - 1s 787us/step - loss: 0.5450 - accuracy: 0.7846
Epoch 10/200
831/831 [==============================] - 1s 776us/step - l